# Question 2

During our living sessions, you studied many different metrics to identify when you have an attack against a system. 

Develop a method that calculates the successful transaction rate for each 30 min interval (use response status code == 200 to identify a  successful transaction ). Following, develop another method that calculates the mean and standard deviation of this successful transaction rate. Using the following heuristic, set a new label for each one of transaction: **set the label suspicious if the transaction is one standard deviation above the mean.**

During the sessions, you saw many different metrics to identify when you have an attack against a system. Develop a method that calculates the success transaction rate (response status code == 200) for 30 min. Following, develop another method that calculates the mean and standard deviation of this successful transaction rate. **Evaluate the transactions using the following heuristic and set a new label  (suspicious) in the successful transactions with one standard deviation above the mean.**

In [1]:
>>> from datetime import timedelta
from datetime import datetime


In [2]:
import pandas as pd


def import_dataset(filename):
    ds = pd.read_csv(filename + '.csv',
                     parse_dates=[3], infer_datetime_format=True)
    ds['datetime'] = ds['datetime'].sort_index()
    return ds

import_dataset("access_log_format-1")

,src_ip,userid,username,datetime,method,response_code,return_obj_size,dest_address,user_agent,country
0,14.139.187.130,hahiss,hahiss@optonline.net,2017-01-01 02:16:51,GET / HTTP/1.1,200,10267,https://www.google.co.in/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,FR
1,14.139.187.130,ahuillet,ahuillet@comcast.net,2017-01-01 02:16:55,GET /GitHub-Mark.png HTTP/1.1,200,7428,http://www.secrepo.com/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,FR
2,14.139.187.130,gtaylor,gtaylor@gmail.com,2017-01-01 02:16:56,GET /twitter-icon.png HTTP/1.1,200,27788,http://www.secrepo.com/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,BR
3,68.180.228.229,terjesa,terjesa@sbcglobal.net,2017-01-01 02:17:59,GET /robots.txt HTTP/1.1,200,233,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...,FR
4,68.180.228.229,smallpaul,smallpaul@optonline.net,2017-01-01 02:17:59,GET /self.logs/access.log.2015-11-01.gz HTTP/1.1,200,6465,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...,US
...,...,...,...,...,...,...,...,...,...,...
7224,85.159.196.242,lipeng,lipeng@att.net,2017-01-04 02:54:55,GET /honeypot/Honeypot%20-%20Howto.pdf HTTP/1.1,206,65874,http://www.secrepo.com/,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:50....,BR
7225,85.159.196.242,peterhoeg,peterhoeg@hotmail.com,2017-01-04 02:54:53,GET /honeypot/Honeypot%20-%20Howto.pdf HTTP/1.1,200,1279418,http://www.secrepo.com/,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:50....,US
7226,85.159.196.242,wikinerd,wikinerd@live.com,2017-01-04 02:54:56,GET /favicon.ico HTTP/1.1,200,267,-,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:50....,BR
7227,85.159.196.242,sblack,sblack@yahoo.ca,2017-01-04 02:54:54,GET /honeypot/Honeypot%20-%20Howto.pdf HTTP/1.1,206,65867,http://www.secrepo.com/,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:50....,MX


In [3]:
ds = import_dataset("access_log_format-1")
ds = ds.dropna()

**Now we get a new dataset that only has the succesful transactions, so the ones where the response status code is equal to 200**

In [4]:
dataset200 = ds[ds["response_code"]== 200]

dataset200.head()

,src_ip,userid,username,datetime,method,response_code,return_obj_size,dest_address,user_agent,country
0,14.139.187.130,hahiss,hahiss@optonline.net,2017-01-01 02:16:51,GET / HTTP/1.1,200,10267,https://www.google.co.in/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,FR
1,14.139.187.130,ahuillet,ahuillet@comcast.net,2017-01-01 02:16:55,GET /GitHub-Mark.png HTTP/1.1,200,7428,http://www.secrepo.com/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,FR
2,14.139.187.130,gtaylor,gtaylor@gmail.com,2017-01-01 02:16:56,GET /twitter-icon.png HTTP/1.1,200,27788,http://www.secrepo.com/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,BR
3,68.180.228.229,terjesa,terjesa@sbcglobal.net,2017-01-01 02:17:59,GET /robots.txt HTTP/1.1,200,233,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...,FR
4,68.180.228.229,smallpaul,smallpaul@optonline.net,2017-01-01 02:17:59,GET /self.logs/access.log.2015-11-01.gz HTTP/1.1,200,6465,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...,US


**To this new dataset now we divide it in 30 minutes intervals and get how many entries we grouped to know how many succesful transactions where in each group, also we previously drop the rows where there we NaN values**

In [5]:
dataset200Resample = dataset200.resample('30min', on='datetime').size()
dataset200Resample.head()

datetime
2017-01-01 02:00:00     5
2017-01-01 02:30:00     0
2017-01-01 03:00:00     2
2017-01-01 03:30:00     0
2017-01-01 04:00:00    11
Freq: 30T, dtype: int64

**We do the same we did in the last step, but this time in the complete dataset in order to obtain how many transactions where in total in each group**

In [6]:
dsResample = ds.resample('30min', on='datetime').size()
dsResample.head()

datetime
2017-01-01 02:00:00     5
2017-01-01 02:30:00     0
2017-01-01 03:00:00     2
2017-01-01 03:30:00     0
2017-01-01 04:00:00    15
Freq: 30T, dtype: int64

**We apply a join to the two grouped dataset so the next steps are easier**

In [7]:
concatDataset = pd.concat([dataset200Resample, dsResample], axis = 1, join="inner")


In [8]:
concatDataset.head()

,0,1
datetime,,
2017-01-01 02:00:00,5,5
2017-01-01 02:30:00,0,0
2017-01-01 03:00:00,2,2
2017-01-01 03:30:00,0,0
2017-01-01 04:00:00,11,15


The ratio is obtained with the formula **ratio = succesful transactions/total transactions** se we create a new column with the results of that formula

In [9]:
concatDataset['ratio'] = concatDataset[0] / concatDataset[1]

In [10]:
concatDataset.head()

,0,1,ratio
datetime,,,
2017-01-01 02:00:00,5,5,1.000000
2017-01-01 02:30:00,0,0,NaN
2017-01-01 03:00:00,2,2,1.000000
2017-01-01 03:30:00,0,0,NaN
2017-01-01 04:00:00,11,15,0.733333


**Now we replace the NaN values for 0 and calculate the mean and the standard deviation**

In [11]:
#concatDataset = concatDataset.fillna(0)
concatDataset = concatDataset.dropna()
concatDataset.head()

,0,1,ratio
datetime,,,
2017-01-01 02:00:00,5,5,1.000000
2017-01-01 03:00:00,2,2,1.000000
2017-01-01 04:00:00,11,15,0.733333
2017-01-01 04:30:00,10,11,0.909091
2017-01-01 05:00:00,7,7,1.000000


In [12]:
concatDataset['mean'] = concatDataset["ratio"].mean()
concatDataset.head()

,0,1,ratio,mean
datetime,,,,
2017-01-01 02:00:00,5,5,1.000000,0.589226
2017-01-01 03:00:00,2,2,1.000000,0.589226
2017-01-01 04:00:00,11,15,0.733333,0.589226
2017-01-01 04:30:00,10,11,0.909091,0.589226
2017-01-01 05:00:00,7,7,1.000000,0.589226


In [13]:
concatDataset['std'] = concatDataset["ratio"].std()
concatDataset.head()

,0,1,ratio,mean,std
datetime,,,,,
2017-01-01 02:00:00,5,5,1.000000,0.589226,0.282244
2017-01-01 03:00:00,2,2,1.000000,0.589226,0.282244
2017-01-01 04:00:00,11,15,0.733333,0.589226,0.282244
2017-01-01 04:30:00,10,11,0.909091,0.589226,0.282244
2017-01-01 05:00:00,7,7,1.000000,0.589226,0.282244


**Now we get the z scores so we know how many standard deviations each group is from the mean**

In [14]:
concatDataset['z'] = (concatDataset["ratio"] - concatDataset["ratio"].mean())/concatDataset["ratio"].std()
concatDataset.head(30)

,0,1,ratio,mean,std,z
datetime,,,,,,
2017-01-01 02:00:00,5,5,1.000000,0.589226,0.282244,1.455385
2017-01-01 03:00:00,2,2,1.000000,0.589226,0.282244,1.455385
2017-01-01 04:00:00,11,15,0.733333,0.589226,0.282244,0.510576
2017-01-01 04:30:00,10,11,0.909091,0.589226,0.282244,1.133291
2017-01-01 05:00:00,7,7,1.000000,0.589226,0.282244,1.455385
2017-01-01 05:30:00,5,8,0.625000,0.589226,0.282244,0.126747
2017-01-01 06:00:00,1,1,1.000000,0.589226,0.282244,1.455385
2017-01-01 06:30:00,8,12,0.666667,0.589226,0.282244,0.274374
2017-01-01 07:00:00,9,11,0.818182,0.589226,0.282244,0.811197


In [15]:
print(concatDataset['z'].max())

1.4553854938419597


**Now we add the tag to those lines were the z is above 1**

In [16]:
import numpy as np

concatDataset['tag'] = np.where(concatDataset['z'] >= 1.0, 'sus', 'normal')
concatDataset.head(30)

,0,1,ratio,mean,std,z,tag
datetime,,,,,,,
2017-01-01 02:00:00,5,5,1.000000,0.589226,0.282244,1.455385,sus
2017-01-01 03:00:00,2,2,1.000000,0.589226,0.282244,1.455385,sus
2017-01-01 04:00:00,11,15,0.733333,0.589226,0.282244,0.510576,normal
2017-01-01 04:30:00,10,11,0.909091,0.589226,0.282244,1.133291,sus
2017-01-01 05:00:00,7,7,1.000000,0.589226,0.282244,1.455385,sus
2017-01-01 05:30:00,5,8,0.625000,0.589226,0.282244,0.126747,normal
2017-01-01 06:00:00,1,1,1.000000,0.589226,0.282244,1.455385,sus
2017-01-01 06:30:00,8,12,0.666667,0.589226,0.282244,0.274374,normal
2017-01-01 07:00:00,9,11,0.818182,0.589226,0.282244,0.811197,normal


**Here are the 30 minutes groups marked as normal or suspicious**